### get svg points array from csv

In [1]:
csv_name = "./isolated.csv"

In [7]:
import numpy as np

csv_path = csv_name


def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=",")
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs


paths = read_csv(csv_path)
print(paths)

[[array([[109.14299774, 125.58000183],
       [110.15747833, 125.49578094],
       [111.17194366, 125.40398407],
       [112.18638611, 125.30522156],
       [113.20079041, 125.20011139],
       [114.21517181, 125.08926392],
       [115.22950745, 124.97328949],
       [116.2437973 , 124.85279083],
       [117.25804138, 124.72838593],
       [118.27222443, 124.60068512],
       [119.28635406, 124.47029877],
       [120.30041504, 124.33782959],
       [121.31440735, 124.2039032 ],
       [122.32831573, 124.06912231],
       [123.34214783, 123.93410492],
       [124.355896  , 123.79945374],
       [125.36954498, 123.66577911],
       [126.38310242, 123.53369141],
       [127.39655304, 123.40380859],
       [128.40989685, 123.27674103],
       [129.42312622, 123.15309143],
       [130.43623352, 123.03347778],
       [131.44923401, 122.91851044],
       [132.46208191, 122.80879974],
       [133.47480774, 122.70494843],
       [134.48739624, 122.60758209],
       [135.49983215, 122.51730347],

In [31]:
import numpy as np
import matplotlib.pyplot as plt

x, y = [], []
for path_XY in paths:
    for XY in path_XY:
        x.append(XY[:, 0])
        y.append(XY[:, 1])

x = np.array(x[0])
y = np.array(y[0])

# plt.plot(x, y)
# plt.title("Curve plotted using the given points")
# plt.xlabel("X")
# plt.ylabel("Y")
# plt.show()

### convert the svg to image

In [32]:
import numpy as np
import numpy as np
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt 

X_Y_Spline = make_interp_spline(x, y)
 
# Returns evenly spaced numbers
# over a specified interval.
X_ = np.linspace(x.min(), x.max(), 500)
Y_ = X_Y_Spline(X_)
 
# Plotting the Graph
plt.plot(X_, Y_)
plt.title("Plot Smooth Curve Using the scipy.interpolate.make_interp_spline() Class")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

ValueError: Expect x to be a 1D strictly increasing sequence.

In [3]:
import svgwrite
import os

os.environ["path"] += r";C:\Program Files\GTK3-Runtime Win64\bin"
import cairosvg


def polylines2svg(
    paths_XYs,
    svg_path,
    colours=["yellow"],
):
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)
    # Create a new SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile="tiny", shape_rendering="crispEdges")
    group = dwg.g()
    for i, path in enumerate(paths_XYs):
        path_data = []
        c = colours[i % len(colours)]
        for XY in path:
            path_data.append(("M", (XY[0, 0], XY[0, 1])))
            for j in range(1, len(XY)):
                path_data.append(("L", (XY[j, 0], XY[j, 1])))
            # if not np.allclose(XY[0], XY[-1]):
            #     path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill="none", stroke=c, stroke_width=2))
    dwg.add(group)
    dwg.save()
    png_path = svg_path.replace(".svg", ".png")
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(
        url=svg_path,
        write_to=png_path,
        parent_width=W,
        parent_height=H,
        output_width=fact * W,
        output_height=fact * H,
        background_color="white",
    )
    return


polylines2svg(paths, "test.svg")

In [4]:
import numpy as np
print(np.__version__)

2.0.1


### import opencv and initialization

In [6]:
import numpy as np
import cv2  # Only for displaying results, not for shape detection

def detect_shapes(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define color thresholds (adjust as needed)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Create masks
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Find contours (connected components)
    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)

        # Calculate area to filter out small contours
        area = cv2.contourArea(cnt)
        if area < 500:  # Adjust threshold as needed
            continue

        if len(approx) == 4:
            shape = "square"
            # Draw a green square border
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        elif len(approx) > 4 and area > 1000:  # Filter out small circles
            shape = "circle"
            # Draw a blue circle border
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)
            cv2.circle(img, center, radius, (255, 0, 0), 2)
        else:
            shape = "unknown"

        # Draw label
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.putText(img, shape, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.namedWindow('Shapes', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Shapes', 600, 600)
    cv2.imshow("Shapes", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = "test.png"
detect_shapes(image_path)
